In [1]:
import pandas as pd
import numpy as np
import json
import requests
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

<a href='https://web.postman.co/workspace/Springboard~8fb65eb5-3826-4258-92f7-fd1474949256/collection/24117868-5badc386-8106-46be-9093-38903d50c5e1?tab=authorization'>Postman collection with explanation of queries used to access desired data</a>

Access the first page of results from <a href='https://api.data.gov/ed/collegescorecard/v1/schools'>api.data.gov</a>
<br>A pipeline for data cleaning will be demonstrated for this page and applied to all pages from the latest collection.
<br>If needed, this process will be repeated for multiple years to effectively resample the data and supplement number of observations.

The information from the school object will be requested separately. The reasoning for this decision is that if it becomes necessary to pull in data from multiple years, these values will not be repeated.

Data descriptions located in references directory, or can be downloaded directly at <a href='https://collegescorecard.ed.gov/data/documentation/'>College Scorecard Webiste</a> and API documentation can be viewed <a href=https://github.com/RTICWDT/open-data-maker/blob/master/API.md>here</a>.

In [3]:
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
response = requests.get(url,
                        params={'fields':'id,latest.student,latest.cost,latest.aid,latest.completion,latest.admission,latest.academics',
                                'keys_nested':'true',
                                'page':0,
                                'api_key':API_KEY}
                       )
res = response.json()

In [4]:
# Removing 'program_available' key, which returns true of false
# Depending on whether 'program' has a non-zero value
# 2 indicates program available distance edu only
data = res['results']
# Serializing json
json_object = json.dumps(data, indent=4)
# Writing to file
with open("../data/raw/page_{}_{}.json".format('latest',str(0)), "w") as outfile:
    outfile.write(json_object)

for datum in data:
    datum['latest']['academics'].pop('program_reporter', None)
    datum['latest']['academics'].pop('program_available', None)
    datum['latest']['cost'].pop('program_reporter', None)

In [5]:
df = pd.json_normalize(data)
df.head()

,id,latest.student.size,latest.student.enrollment.all,latest.student.enrollment.undergrad_12_month,latest.student.enrollment.grad_12_month,latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.hispanic,latest.student.demographics.race_ethnicity.asian,latest.student.demographics.race_ethnicity.aian,...,latest.academics.program.degree_or_certificate.public_administration_social_service,latest.academics.program.degree_or_certificate.social_science,latest.academics.program.degree_or_certificate.construction,latest.academics.program.degree_or_certificate.mechanic_repair_technology,latest.academics.program.degree_or_certificate.precision_production,latest.academics.program.degree_or_certificate.transportation,latest.academics.program.degree_or_certificate.visual_performing,latest.academics.program.degree_or_certificate.health,latest.academics.program.degree_or_certificate.business_marketing,latest.academics.program.degree_or_certificate.history
0,100654,5090,None,5479,1081.0,0.0159,0.9022,0.0116,0.0012,0.0028,...,1,1,0,0,0,0,1,0,1,0
1,100663,13549,None,14969,10874.0,0.5496,0.2401,0.0610,0.0704,0.0024,...,1,2,0,0,0,0,1,1,1,1
2,100690,298,None,472,607.0,0.2550,0.6913,0.0268,0.0034,0.0000,...,0,0,0,0,0,0,0,0,2,0
3,100706,7825,None,8898,2414.0,0.7173,0.0907,0.0599,0.0354,0.0083,...,0,1,0,0,0,0,1,1,1,1
4,100724,3603,None,4127,513.0,0.0167,0.9265,0.0130,0.0019,0.0017,...,1,1,0,0,0,0,1,1,1,1


In [6]:
len(df.isna().sum()[df.isna().sum() > 0])

1621

The data dictionary indicates that certain features are discontinued. These columns are occupied with complete columns of `None` values, and will be dropped.
<br>*Note:* Many dropped columns are availabe in 2000-2001 data only. Be sure to not include data from this year if using multiple years to 'resample'.

In [7]:
df_dropping = df.copy()
df_dropping.head()

,id,latest.student.size,latest.student.enrollment.all,latest.student.enrollment.undergrad_12_month,latest.student.enrollment.grad_12_month,latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.hispanic,latest.student.demographics.race_ethnicity.asian,latest.student.demographics.race_ethnicity.aian,...,latest.academics.program.degree_or_certificate.public_administration_social_service,latest.academics.program.degree_or_certificate.social_science,latest.academics.program.degree_or_certificate.construction,latest.academics.program.degree_or_certificate.mechanic_repair_technology,latest.academics.program.degree_or_certificate.precision_production,latest.academics.program.degree_or_certificate.transportation,latest.academics.program.degree_or_certificate.visual_performing,latest.academics.program.degree_or_certificate.health,latest.academics.program.degree_or_certificate.business_marketing,latest.academics.program.degree_or_certificate.history
0,100654,5090,None,5479,1081.0,0.0159,0.9022,0.0116,0.0012,0.0028,...,1,1,0,0,0,0,1,0,1,0
1,100663,13549,None,14969,10874.0,0.5496,0.2401,0.0610,0.0704,0.0024,...,1,2,0,0,0,0,1,1,1,1
2,100690,298,None,472,607.0,0.2550,0.6913,0.0268,0.0034,0.0000,...,0,0,0,0,0,0,0,0,2,0
3,100706,7825,None,8898,2414.0,0.7173,0.0907,0.0599,0.0354,0.0083,...,0,1,0,0,0,0,1,1,1,1
4,100724,3603,None,4127,513.0,0.0167,0.9265,0.0130,0.0019,0.0017,...,1,1,0,0,0,0,1,1,1,1


In [8]:
# We will probably refer back to this subsetting method frequently
completion_columns = [feat for feat in df_dropping.columns if 'latest.completion' in feat]
zero_repl = dict(zip(completion_columns, np.zeros_like(completion_columns,dtype='int')))

df_dropping = df_dropping.fillna(zero_repl)
df_dropping.head()

,id,latest.student.size,latest.student.enrollment.all,latest.student.enrollment.undergrad_12_month,latest.student.enrollment.grad_12_month,latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.hispanic,latest.student.demographics.race_ethnicity.asian,latest.student.demographics.race_ethnicity.aian,...,latest.academics.program.degree_or_certificate.public_administration_social_service,latest.academics.program.degree_or_certificate.social_science,latest.academics.program.degree_or_certificate.construction,latest.academics.program.degree_or_certificate.mechanic_repair_technology,latest.academics.program.degree_or_certificate.precision_production,latest.academics.program.degree_or_certificate.transportation,latest.academics.program.degree_or_certificate.visual_performing,latest.academics.program.degree_or_certificate.health,latest.academics.program.degree_or_certificate.business_marketing,latest.academics.program.degree_or_certificate.history
0,100654,5090,None,5479,1081.0,0.0159,0.9022,0.0116,0.0012,0.0028,...,1,1,0,0,0,0,1,0,1,0
1,100663,13549,None,14969,10874.0,0.5496,0.2401,0.0610,0.0704,0.0024,...,1,2,0,0,0,0,1,1,1,1
2,100690,298,None,472,607.0,0.2550,0.6913,0.0268,0.0034,0.0000,...,0,0,0,0,0,0,0,0,2,0
3,100706,7825,None,8898,2414.0,0.7173,0.0907,0.0599,0.0354,0.0083,...,0,1,0,0,0,0,1,1,1,1
4,100724,3603,None,4127,513.0,0.0167,0.9265,0.0130,0.0019,0.0017,...,1,1,0,0,0,0,1,1,1,1


SAT writing section no longer exists. It seems some scores may have been presented for students who enrolled before this recent change. Columns related to SAT writing should be dropped.

In [9]:
writing_columns = [feat for feat in df_dropping.columns if 'writing' in feat]
df_dropping = df_dropping.drop(columns=writing_columns)

Student share data is encoded in much the same way as completion stats. `Null` values indicate that these features do not occupy the student share. 

In [10]:
share_columns = [feat for feat in df_dropping.columns if 'latest.student' in feat]
zero_repl = dict(zip(share_columns, np.zeros_like(share_columns,dtype='int')))

df_dropping = df_dropping.fillna(zero_repl)
df_dropping.head()

,id,latest.student.size,latest.student.enrollment.all,latest.student.enrollment.undergrad_12_month,latest.student.enrollment.grad_12_month,latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.hispanic,latest.student.demographics.race_ethnicity.asian,latest.student.demographics.race_ethnicity.aian,...,latest.academics.program.degree_or_certificate.public_administration_social_service,latest.academics.program.degree_or_certificate.social_science,latest.academics.program.degree_or_certificate.construction,latest.academics.program.degree_or_certificate.mechanic_repair_technology,latest.academics.program.degree_or_certificate.precision_production,latest.academics.program.degree_or_certificate.transportation,latest.academics.program.degree_or_certificate.visual_performing,latest.academics.program.degree_or_certificate.health,latest.academics.program.degree_or_certificate.business_marketing,latest.academics.program.degree_or_certificate.history
0,100654,5090,0,5479,1081.0,0.0159,0.9022,0.0116,0.0012,0.0028,...,1,1,0,0,0,0,1,0,1,0
1,100663,13549,0,14969,10874.0,0.5496,0.2401,0.0610,0.0704,0.0024,...,1,2,0,0,0,0,1,1,1,1
2,100690,298,0,472,607.0,0.2550,0.6913,0.0268,0.0034,0.0000,...,0,0,0,0,0,0,0,0,2,0
3,100706,7825,0,8898,2414.0,0.7173,0.0907,0.0599,0.0354,0.0083,...,0,1,0,0,0,0,1,1,1,1
4,100724,3603,0,4127,513.0,0.0167,0.9265,0.0130,0.0019,0.0017,...,1,1,0,0,0,0,1,1,1,1


In [11]:
# enrollment.all is discontinued
df_dropping = df_dropping.drop(columns='latest.student.enrollment.all')

Strange formatting for public private universties. Due to the structure of the JSON, private universities have `null` values for all public cost fields, and vice versa. This unconventional fill method should take care of this issue.

In [12]:
public_columns = [
    feat for feat in df_dropping.columns 
    if 'latest.cost.avg_net_price.public' in feat
    or 'latest.cost.net_price.public' in feat
    or 'latest.cost.title_iv.public' in feat
]
private_columns = [
    feat for feat in df_dropping.columns 
    if 'latest.cost.avg_net_price.private' in feat
    or 'latest.cost.net_price.private' in feat
    or 'latest.cost.title_iv.private' in feat
]
program_columns = [
    feat for feat in df_dropping.columns 
    if 'latest.cost.avg_net_price.program_year' in feat
    or 'latest.cost.net_price.program_reporter' in feat
    or 'latest.cost.title_iv.program_reporter' in feat
    or 'latest.cost.title_iv.program_year' in feat
]
other_ay_columns = [
    feat for feat in df_dropping.columns 
    if 'latest.cost.avg_net_price.other_academic_year' in feat
    or 'latest.cost.net_price.other_acad_calendar' in feat
    or 'latest.cost.title_iv.other_acad_calendar' in feat
    or 'latest.cost.title_iv.academic_year' in feat
]
for i in range(len(public_columns)):
    df_dropping[public_columns[i]].fillna(df_dropping[private_columns[i]],inplace=True)
    df_dropping[public_columns[i]].fillna(df_dropping[program_columns[i]],inplace=True)
    df_dropping[public_columns[i]].fillna(df_dropping[other_ay_columns[i]],inplace=True)
df_dropping = df_dropping.drop(columns=private_columns)
df_dropping = df_dropping.drop(columns=program_columns)
df_dropping = df_dropping.drop(columns=other_ay_columns)


In [13]:
df_dropping.columns = df_dropping.columns.str.replace(r'latest.cost.avg_net_price.public',
                                                      'latest.cost.avg_net_price',
                                                      regex=True)
df_dropping.columns = df_dropping.columns.str.replace(r'latest.cost.net_price.public',
                                                      'latest.cost.net_price',
                                                      regex=True)
df_dropping.columns = df_dropping.columns.str.replace(r'latest.cost.title_iv.public',
                                                      'latest.cost.title_iv',
                                                      regex=True)

A similar pattern as above makes the data for cost spare by program-year institutions vs. academic year institutions. We will do a similar fill here, and simplify the attendance costs into 1 column. Program year tuition expenses will be placed in in-state and out-of-state tuition columns on the assumption that certificate program granting institutions don't have separate expenses by state.

In [14]:
df_dropping['latest.cost.attendance.academic_year'].fillna(
    df_dropping['latest.cost.attendance.program_year'],
    inplace=True)
df_dropping['latest.cost.tuition.in_state'].fillna(
    df_dropping['latest.cost.tuition.program_year'],
    inplace=True)
df_dropping['latest.cost.tuition.out_of_state'].fillna(
    df_dropping['latest.cost.tuition.program_year'],
    inplace=True)
df_dropping = df_dropping.drop(columns=['latest.cost.attendance.program_year', 
                                        'latest.cost.tuition.program_year'])
# The distinction between academic and program year and vague,
# so I will not bother renaming this column

In [15]:
len(df_dropping.isna().sum()[df_dropping.isna().sum() > 0])

154

In [16]:
df_renaming = df_dropping.copy()
df_renaming.columns = df_renaming.columns.str.replace(r'latest.','',regex=True)
df_renaming.columns = df_renaming.columns.str.replace(r'.','_',regex=True)
df_renaming.head()

,id,student_size,student_enrollment_undergrad_12_month,student_enrollment_grad_12_month,student_demographics_race_ethnicity_white,student_demographics_race_ethnicity_black,student_demographics_race_ethnicity_hispanic,student_demographics_race_ethnicity_asian,student_demographics_race_ethnicity_aian,student_demographics_race_ethnicity_nhpi,...,academics_program_degree_or_certificate_public_administration_social_service,academics_program_degree_or_certificate_social_science,academics_program_degree_or_certificate_construction,academics_program_degree_or_certificate_mechanic_repair_technology,academics_program_degree_or_certificate_precision_production,academics_program_degree_or_certificate_transportation,academics_program_degree_or_certificate_visual_performing,academics_program_degree_or_certificate_health,academics_program_degree_or_certificate_business_marketing,academics_program_degree_or_certificate_history
0,100654,5090,5479,1081.0,0.0159,0.9022,0.0116,0.0012,0.0028,0.0008,...,1,1,0,0,0,0,1,0,1,0
1,100663,13549,14969,10874.0,0.5496,0.2401,0.0610,0.0704,0.0024,0.0004,...,1,2,0,0,0,0,1,1,1,1
2,100690,298,472,607.0,0.2550,0.6913,0.0268,0.0034,0.0000,0.0000,...,0,0,0,0,0,0,0,0,2,0
3,100706,7825,8898,2414.0,0.7173,0.0907,0.0599,0.0354,0.0083,0.0010,...,0,1,0,0,0,0,1,1,1,1
4,100724,3603,4127,513.0,0.0167,0.9265,0.0130,0.0019,0.0017,0.0017,...,1,1,0,0,0,0,1,1,1,1


Sans EDA, we will consider how to handle these missing values later. Let's package these steps into a function and use a new notebook to repeat this process for each page. But first, let's take a look at the school data and make sure there's nothing weird going on there. We will ultimately want to join this to the above data.

In [17]:
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
response = requests.get(url,
                        params={'fields':'id,school',
                                'keys_nested':'true',
                                'page':0,
                                'api_key':API_KEY}
                       )
res = response.json()

data = res['results']
# Serializing json
json_object = json.dumps(data, indent=4)
# Writing to file
with open("../data/raw/schools_page_{}.json".format(str(0)), "w") as outfile:
    outfile.write(json_object)
    
df_school_data = pd.json_normalize(data)
df_school_data.head()

,id,school.name,school.city,school.state,school.zip,school.accreditor,school.school_url,school.price_calculator_url,school.degrees_awarded.predominant_recoded,school.degrees_awarded.predominant,...,school.institutional_characteristics.level,school.open_admissions_policy,school.accreditor_code,school.title_iv.approval_date,school.title_iv.eligibility_type,school.ownership_peps,school.endowment.begin,school.endowment.end,school.dolflag,school.search
0,100654,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,3,3,...,1,2.0,SACSCC,12/12/1965,1,1,NaN,NaN,0,None
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,3,3,...,1,2.0,SACSCC,12/1/1965,1,1,537349307.0,539858544.0,0,None
2,100690,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,3,2,...,1,1.0,SACSCC,3/26/1987,1,2,174805.0,174818.0,0,None
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,3,3,...,1,2.0,SACSCC,12/1/1965,1,1,77250279.0,75837207.0,1,None
4,100724,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,3,3,...,1,2.0,SACSCC,12/1/1965,1,1,94536751.0,111315175.0,0,None


In [18]:
# Some of these columns can definitely be dropped
df_school_data.columns = df_school_data.columns.str.replace(r'school.','',regex=True)
df_school_data.columns = df_school_data.columns.str.replace(r'.','_',regex=True)
df_school_data_drops = df_school_data.drop(columns=['zip','url','price_calculator_url',
                                                    'search','accreditor','alias','degree_urbanization'])
df_school_data_drops.head()

,id,name,city,state,degrees_awarded_predominant_recoded,degrees_awarded_predominant,degrees_awarded_highest,under_investigation,main_campus,branches,...,ft_faculty_rate,institutional_characteristics_level,open_admissions_policy,accreditor_code,title_iv_approval_date,title_iv_eligibility_type,ownership_peps,endowment_begin,endowment_end,dolflag
0,100654,Alabama A & M University,Normal,AL,3,3,4,0,1,1,...,0.9960,1,2.0,SACSCC,12/12/1965,1,1,NaN,NaN,0
1,100663,University of Alabama at Birmingham,Birmingham,AL,3,3,4,0,1,1,...,0.7619,1,2.0,SACSCC,12/1/1965,1,1,537349307.0,539858544.0,0
2,100690,Amridge University,Montgomery,AL,3,2,4,0,1,1,...,1.0000,1,1.0,SACSCC,3/26/1987,1,2,174805.0,174818.0,0
3,100706,University of Alabama in Huntsville,Huntsville,AL,3,3,4,0,1,1,...,0.6702,1,2.0,SACSCC,12/1/1965,1,1,77250279.0,75837207.0,1
4,100724,Alabama State University,Montgomery,AL,3,3,4,0,1,1,...,0.6797,1,2.0,SACSCC,12/1/1965,1,1,94536751.0,111315175.0,0


In [19]:
df_school_data_drops = df_school_data_drops.fillna({'religious_affiliation':0})

In [20]:
print(len(df_school_data_drops.isna().sum()[df_school_data_drops.isna().sum() > 0]))
df_school_data_drops.isna().sum()[df_school_data_drops.isna().sum() > 0]

5


faculty_salary            1
ft_faculty_rate           1
open_admissions_policy    1
endowment_begin           5
endowment_end             5
dtype: int64

In [21]:
schools_df = df_school_data_drops.copy()
df_info = df_renaming.copy()
df = schools_df.merge(df_info,on='id')
print(schools_df.shape)
print(df_info.shape)
print(df.shape)
df.head()

(20, 42)
(20, 1938)
(20, 1979)


,id,name,city,state,degrees_awarded_predominant_recoded,degrees_awarded_predominant,degrees_awarded_highest,under_investigation,main_campus,branches,...,academics_program_degree_or_certificate_public_administration_social_service,academics_program_degree_or_certificate_social_science,academics_program_degree_or_certificate_construction,academics_program_degree_or_certificate_mechanic_repair_technology,academics_program_degree_or_certificate_precision_production,academics_program_degree_or_certificate_transportation,academics_program_degree_or_certificate_visual_performing,academics_program_degree_or_certificate_health,academics_program_degree_or_certificate_business_marketing,academics_program_degree_or_certificate_history
0,100654,Alabama A & M University,Normal,AL,3,3,4,0,1,1,...,1,1,0,0,0,0,1,0,1,0
1,100663,University of Alabama at Birmingham,Birmingham,AL,3,3,4,0,1,1,...,1,2,0,0,0,0,1,1,1,1
2,100690,Amridge University,Montgomery,AL,3,2,4,0,1,1,...,0,0,0,0,0,0,0,0,2,0
3,100706,University of Alabama in Huntsville,Huntsville,AL,3,3,4,0,1,1,...,0,1,0,0,0,0,1,1,1,1
4,100724,Alabama State University,Montgomery,AL,3,3,4,0,1,1,...,1,1,0,0,0,0,1,1,1,1
